## Internal Control Variables (ICVs)

According to Section 2.3 of the OpenMP 4.0 specification, an OpenMP implementation must act as if there are ICVs that control  the behavior of the program.  This example illustrates two ICVs,  _nthreads-var_   and  _max-active-levels-var_ . The  _nthreads-var_  ICV controls the  number of threads requested for encountered parallel regions; there is one copy  of this ICV per task. The  _max-active-levels-var_  ICV controls the maximum  number of nested active parallel regions; there is one copy of this ICV for the  whole program.

In the following example, the  _nest-var_ ,  _max-active-levels-var_ ,   _dyn-var_ , and  _nthreads-var_  ICVs are modified through calls to  the runtime library routines __omp_set_nested__, __omp_set_max_active_levels__, __omp_set_dynamic__, and __omp_set_num_threads__ respectively. These ICVs  affect the operation of __parallel__ regions. Each implicit task generated  by a __parallel__ region has its own copy of the  _nest-var, dyn-var_ ,  and  _nthreads-var_  ICVs.

In the following example, the new value of  _nthreads-var_  applies only to  the implicit tasks that execute the call to __omp_set_num_threads__. There  is one copy of the  _max-active-levels-var_  ICV for the whole program and  its value is the same for all tasks. This example assumes that nested parallelism  is supported.

The outer __parallel__ region creates a team of two threads; each of the threads  will execute one of the two implicit tasks generated by the outer __parallel__  region.

Each implicit task generated by the outer __parallel__ region calls __omp_set_num_threads(3)__,  assigning the value 3 to its respective copy of  _nthreads-var_ . Then each  implicit task encounters an inner __parallel__ region that creates a team  of three threads; each of the threads will execute one of the three implicit tasks  generated by that inner __parallel__ region.

Since the outer __parallel__ region is executed by 2 threads, and the inner  by 3, there will be a total of 6 implicit tasks generated by the two inner __parallel__  regions.

Each implicit task generated by an inner __parallel__ region will execute  the call to __omp_set_num_threads(4)__, assigning the value 4 to its respective  copy of  _nthreads-var_ .

The print statement in the outer __parallel__ region is executed by only one  of the threads in the team. So it will be executed only once.

The print statement in an inner __parallel__ region is also executed by only  one of the threads in the team. Since we have a total of two inner __parallel__  regions, the print statement will be executed twice - once per inner __parallel__  region.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: icv.1
* type: C
*/
#include <stdio.h>
#include <omp.h>

int main (void)
{
  omp_set_nested(1);
  omp_set_max_active_levels(8);
  omp_set_dynamic(0);
  omp_set_num_threads(2);
  #pragma omp parallel
    {
      omp_set_num_threads(3);

      #pragma omp parallel
        {
          omp_set_num_threads(4);
          #pragma omp single
            {
                 // The following should print:
                 // Inner: max_act_lev=8, num_thds=3, max_thds=4
                 // Inner: max_act_lev=8, num_thds=3, max_thds=4
              printf ("Inner: max_act_lev=%d, num_thds=%d, max_thds=%d\n",
              omp_get_max_active_levels(), omp_get_num_threads(),
              omp_get_max_threads());
            }
        }

      #pragma omp barrier
      #pragma omp single
        {
                 // The following should print:
                 // Outer: max_act_lev=8, num_thds=2, max_thds=3
          printf ("Outer: max_act_lev=%d, num_thds=%d, max_thds=%d\n",
                  omp_get_max_active_levels(), omp_get_num_threads(),
                  omp_get_max_threads());
        }
    }
    return 0;
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: icv.1
! type: F-fixed
      program icv
      use omp_lib

      call omp_set_nested(.true.)
      call omp_set_max_active_levels(8)
      call omp_set_dynamic(.false.)
      call omp_set_num_threads(2)

!$omp parallel
      call omp_set_num_threads(3)

!$omp parallel
      call omp_set_num_threads(4)
!$omp single
!      The following should print:
!      Inner: max_act_lev= 8 , num_thds= 3 , max_thds= 4
!      Inner: max_act_lev= 8 , num_thds= 3 , max_thds= 4
       print *, "Inner: max_act_lev=", omp_get_max_active_levels(),
     &           ", num_thds=", omp_get_num_threads(),
     &           ", max_thds=", omp_get_max_threads()
!$omp end single
!$omp end parallel

!$omp barrier
!$omp single
!      The following should print:
!      Outer: max_act_lev= 8 , num_thds= 2 , max_thds= 3
       print *, "Outer: max_act_lev=", omp_get_max_active_levels(),
     &           ", num_thds=", omp_get_num_threads(),
     &           ", max_thds=", omp_get_max_threads()
!$omp end single
!$omp end parallel
       end